In [1]:
# Transformers installation
! pip install transformers datasets evaluate seqeval accelerate
# To install from source instead of the last release, comment the command above and uncomment the following one.
# ! pip install git+https://github.com/huggingface/transformers.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 34.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 71.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=1

In [100]:
!pip install torch

In [102]:
import torch

In [38]:
from huggingface_hub import notebook_login

notebook_login()

In [85]:
#Loading Data
import json
with open("main.json", encoding='utf-8') as fp:
    dataset = json.load(fp);

'''for i in range(len(dataset)):
    for j in range(len(dataset[i]['tokens'])):
        dataset[i]['tokens'][j] = dataset[i]['tokens'][j].upper()'''
dataset[0]

{'id': '0',
 'ner_tags': ['B-no', 'B-method', 'B-no', 'B-method', 'B-class', 'B-no'],
 'tokens': ['3.1.1', 'Start', 'and', 'Stop', 'Diagnostics', 'Use Case']}

In [93]:
weights = {
    "O":0,
    "B-no":0,
    "I-no":0,
    "B-class":0,
    "I-class":0,
    "B-method":0,
    "I-method":0,
    "B-attribute":0,
    "I-attribute":0,
    "B-association":0,
    "I-association":0,
    "B-generalization":0,
    "I-generalization":0,
}
total=0

for i in dataset:
    for j in i['ner_tags']:
        weights[j]+=1
        total+=1

for k in weights:
    weights[k] /= total

In [116]:
weight_list = list(weights.values())
weight_list = [1-x for x in weight_list]

In [117]:
weight_list

[0.45150620391435536,
 0.8906368491537183,
 0.8181583490649758,
 0.9210513577617503,
 0.9949658744372912,
 0.9610823369959823,
 0.9949981444729497,
 0.9788469916259257,
 0.9987091985736645,
 0.9906578246768962,
 0.9996127595720993,
 0.9997741097503913,
 1.0]

In [41]:

label2id = {
    "O":0,
    "B-no":1,
    "I-no":2,
    "B-class":3,
    "I-class":4,
    "B-method":5,
    "I-method":6,
    "B-attribute":7,
    "I-attribute":8,
    "B-association":9,
    "I-association":10,
    "B-generalization":11,
    "I-generalization":12,
}

id2label = {
    0:"O",
    1:"B-no",
    2:"I-no",
    3:"B-class",
    4:"I-class",
    5:"B-method",
    6:"I-method",
    7:"B-attribute",
    8:"I-attribute",
    9:"B-association",
    10:"I-association",
    11:"B-generalization",
    12:"I-generalization",
}

In [42]:
for i in range(len(dataset)):
    dataset[i]["ner_tags"] = [label2id[j] for j in dataset[i]["ner_tags"]]

dataset[0]

{'id': '0',
 'ner_tags': [1, 5, 1, 5, 3, 1],
 'tokens': ['3.1.1', 'Start', 'and', 'Stop', 'Diagnostics', 'Use Case']}

In [43]:
from sklearn.model_selection import train_test_split

train_set, test_set = train_test_split(dataset,test_size=0.15)
print(len(train_set),len(test_set))

3725 658


In [44]:
from datasets import IterableDataset

def gen_train():
    for i in train_set:
        yield i

def gen_test():
    for i in test_set:
        yield i

train_dataset = IterableDataset.from_generator(gen_train)
test_dataset = IterableDataset.from_generator(gen_test)

In [45]:
label_list = list(label2id.keys())
label_list

['O',
 'B-no',
 'I-no',
 'B-class',
 'I-class',
 'B-method',
 'I-method',
 'B-attribute',
 'I-attribute',
 'B-association',
 'I-association',
 'B-generalization',
 'I-generalization']

In [63]:
pretrained_model = "bert-base-uncased"

In [64]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

In [65]:
# example = wnut["train"][0]
example = dataset[0]
print(example)
tokenized_input = tokenizer(example["tokens"], is_split_into_words=True)
tokens = tokenizer.convert_ids_to_tokens(tokenized_input["input_ids"])
tokens

{'id': '0', 'ner_tags': [1, 5, 1, 5, 3, 1], 'tokens': ['3.1.1', 'Start', 'and', 'Stop', 'Diagnostics', 'Use Case']}


['[CLS]',
 '3',
 '.',
 '1',
 '.',
 '1',
 'start',
 'and',
 'stop',
 'diagnostic',
 '##s',
 'use',
 'case',
 '[SEP]']

In [49]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [50]:
# tokenized_dataset = list(map(tokenize_and_align_labels, dataset))
tokenized_train_dataset = train_dataset.map(tokenize_and_align_labels, batched=True)
tokenized_test_dataset = test_dataset.map(tokenize_and_align_labels, batched=True)

In [51]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [52]:
import evaluate

seqeval = evaluate.load("seqeval")

In [53]:
import numpy as np

labels = [label_list[i] for i in example[f"ner_tags"]]


def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [54]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    pretrained_model, num_labels=13, id2label=id2label, label2id=label2id)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [118]:
from torch import nn

class CustomTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get('logits')
        # compute custom loss
        loss_fct = nn.CrossEntropyLoss(weight=torch.tensor(weight_list, device='cuda:0'))
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))
        return (loss, outputs) if return_outputs else loss

In [119]:
training_args = TrainingArguments(
    output_dir="model",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    max_steps = 1000,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub = True
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_test_dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)


In [120]:
trainer.train()

Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
0,No log,0.790903,0.494527,0.688789,0.575712,0.768664
1,No log,0.747188,0.503829,0.708072,0.588740,0.781748
2,0.342200,0.699019,0.559298,0.700000,0.621788,0.813634
3,0.342200,0.711896,0.580210,0.694170,0.632095,0.823529
4,0.299300,0.714156,0.566375,0.696413,0.624698,0.819791


/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-sco

TrainOutput(global_step=1000, training_loss=0.32076292419433594, metrics={'train_runtime': 332.5598, 'train_samples_per_second': 48.112, 'train_steps_per_second': 3.007, 'total_flos': 683089483301664.0, 'train_loss': 0.32076292419433594, 'epoch': 4.07})

In [121]:
from transformers import pipeline

classifier = pipeline(task="ner",device='cuda:0',model=trainer.model,tokenizer=trainer.tokenizer,aggregation_strategy = "simple")
text = "The device beeps and the message &quot;INVALID INPUT&quot is displayed on the screen ."
classifier(text)

[{'entity_group': 'class',
  'score': 0.9972832,
  'word': 'device',
  'start': 4,
  'end': 10},
 {'entity_group': 'method',
  'score': 0.5284407,
  'word': 'displayed',
  'start': 61,
  'end': 70}]

In [122]:
trainer.push_to_hub()

'https://huggingface.co/suyashmittal/model/tree/main/'